# Crosstab and Groupby

In [1]:
# Don't change this cell; just run it.
import numpy as np  # The array library.
import pandas as pd
# Safe setting for Pandas.  Needs Pandas version >= 1.5.
pd.set_option('mode.copy_on_write', True)

# The OKpy testing system.
from client.api.notebook import Notebook
ok = Notebook('crosstab_groupby.ok')

## The Chronic Kidney Disease dataset

We will use a dataset on chronic kidney disease (CKD) dataset.  Each row in the
data set represents a single patient who was treated in the past and whose
diagnosis is known.

If you are running on your laptop, you should download the `ckd_clean.csv`
dataset via the [chronic kidney disease page](https://lisds.github.io/textbook/data/chronic_kidney_disease).

Load the `ckd_clean.csv` dataset as the variable `ckd`:

In [2]:
ckd = pd.read_csv('ckd_clean.csv')
ckd.head()

In [3]:
_ = ok.grade('q_01_ckd')

The values in the `Class` column are not very evocative: 1 for chronic kidney
disease, 0 for not chronic kidney disease.

Make a new column called `Patient` in the `ckd` data frame.   It should have
`CKD` for patients with chronic kidney disease and `Not CKD` otherwise.

In [4]:
ckd['Patient'] = 'Not CKD'
ckd.loc[ckd['Class'] == 1, 'Patient'] = 'CKD'
# Show the first 5 rows of the result.
ckd.head()

In [5]:
_ = ok.grade('q_02_patient')

We are interested to see the counts of CKD patient and not-patients that have
hypertension (high blood pressure).

Set `ht_by_patient` to be a data frame that is the cross-tabulation of the
`Hypertension` (rows) and `Patient` (columns) categories:

In [6]:
ht_by_patient = pd.crosstab(ckd['Hypertension'], ckd['Patient'])
# Show the resulting table.
ht_by_patient

In [7]:
_ = ok.grade('q_03_ht_by_patient')

Actually, it would be even more useful to see the proportions of CKD and
not-CKD patients that have hypertension.  Set the variable `ht_p_by_patient` to
be a data frame where the values are the proportion of patients in the given
category, that have hypertension.

In [8]:
ht_p_by_patient = pd.crosstab(ckd['Hypertension'], ckd['Patient'],
                              normalize='columns')
# Show the result
ht_p_by_patient

In [9]:
_ = ok.grade('q_04_ht_p_by_patient')

Now we are interested in the incidence of pedal edema in the patients and
not-patients.  Pedal edema is swelling of the feet caused by fluid retention;
it is relatively common in kidney disease.

Now do a multiway table with all the unique pairings of `Hypertension` and
`Pedal Edema` over the rows, and the `Patient` category in the columns.

You will see the following combinations listed in the rows:

* Hypertension: no; pedal edema: no
* Hypertension: no; pedal edema: yes
* Hypertension: yes; pedal edema: no
* Hypertension: yes; pedal edema: yes

Set the variable `ht_pe_by_patient` to contain this data frame.

In [10]:
ht_pe_by_patient = pd.crosstab(
        [ckd['Hypertension'], ckd['Pedal Edema']],
        ckd['Patient'])
# Show the result
ht_pe_by_patient

In [11]:
_ = ok.grade('q_05_ht_pe_by_patient')

## Grouping

Now we are interested in doing some aggregation by categories.

In particular, we want to aggregate by the `Patient` column.

Create a GroupBy object by grouping the `ckd` data frame by the `Patient`
column.  Store this in the variable `by_patient`.

In [12]:
by_patient = ckd.groupby('Patient')
# Show the result
by_patient

In [13]:
_ = ok.grade('q_06_by_patient')

Calculate the means of all the numeric variables over the groups, store the
results as `mean_by_patient`.

In [14]:
mean_by_patient = by_patient.mean()
# Show the result
mean_by_patient

In [15]:
_ = ok.grade('q_07_mean_by_patient')

Actually, we are only interested, for now, in the mean for the `Hemoglobin`
column.  Calculate this mean, broken down by `Patient`.  Store as
`hgb_by_patient`.

In [16]:
hgb_by_patient = by_patient['Hemoglobin'].mean()
# Show the result
hgb_by_patient

In [17]:
_ = ok.grade('q_08_hgb_by_patient')

Use `.loc` indexing to print the results of the same calculation.  You should
select the `CKD` rows from `ckd` and calculate the mean of the `Hemoglobin`
values, and then do the same for the `Not CKD` rows.

In [18]:
print(ckd.loc[ckd['Patient'] == 'CKD', 'Hemoglobin'].mean())
print(ckd.loc[ckd['Patient'] == 'Not CKD', 'Hemoglobin'].mean())

You should find that the results are the same as the results from the GroupBy
above.

Make a GroupBy that breaks down the rows by *both* `Patient` and `Diabetes
Mellitus`.  Show the *median* of all the numerical values for each subgroup:

In [19]:
by_pt_dm = ckd.groupby(['Patient', 'Diabetes Mellitus'])
pt_dm_medians = by_pt_dm.median()
# Show the result
pt_dm_medians

In [20]:
_ = ok.grade('q_09_pt_dm_medians')

Finally we return to the GroupBy over `Patient` (only).  We do want to
aggregate, but we want only two aggregate values.  We want the mean of the
`Blood Pressure` values - we will call the resulting column `mean_bp`.  We also
want the median of the `Age` values.   We will call this column `median_age`:

In [21]:
fancy_values = by_patient.aggregate(
    mean_bp=('Blood Pressure', 'mean'),
    median_age=('Age', 'median')
)
# Show the result
fancy_values

In [22]:
_ = ok.grade('q_10_fancy_values')

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [23]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]